In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import glob
import shutil
import sys
import numpy as np
# from utils import *
# from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint

In [ ]:
data_dir = '../input/emogis/normalized'

In [ ]:
height=28
width=28
batch_size=32
num_classes=7
number_of_classes=7

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=30,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.1,
      horizontal_flip=False,
      fill_mode='nearest',
    validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(height, width),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True)
validation_generator = datagen.flow_from_directory(
    data_dir, 
    target_size=(height, width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True) 

In [ ]:
# model=models.Sequential()
# model.add(layers.Conv2D(32,(5,5),padding='valid',activation='relu',input_shape=(28,28,1)))
# model.add(layers.Conv2D(64,(5,5),padding='valid',activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(128,(3,3),activation='relu'))
# model.add(layers.Conv2D(256,(3,3),activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Dropout(0.25))
# model.add(layers.Flatten())
# model.add(layers.Dense(512,activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(256,activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(num_classes,activation='softmax'))

model=models.Sequential()
model.add(layers.BatchNormalization(input_shape=(28,28,1)))
model.add(layers.Conv2D(32,5,input_shape=(28,28,1),activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128,3,input_shape=(28,28,1),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.BatchNormalization())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(num_classes,activation='softmax'))


# model = models.Sequential([ 
#     layers.BatchNormalization(),
#     layers.Conv2D(32,3,input_shape=(28,28,1)),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D(2,2),
#     layers.BatchNormalization(),
#     layers.Flatten(input_shape=(28,28,1)),
#     layers.Dense(512,activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dense(128,activation='relu'),
#     layers.BatchNormalization(),
#     layers.Dense(number_of_classes,activation='softmax')
# ])

model.summary()

In [ ]:
# optimizer_name = 'adam'

# model.compile(loss='categorical_crossentropy', optimizer=optimizer_name, metrics=['accuracy'])

# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
# mcp_save = ModelCheckpoint('my_model.h5', save_best_only=True, monitor='val_loss', verbose=1, mode='auto')

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

MCP = ModelCheckpoint('Emogis_model.h5',verbose=1,save_best_only=True,monitor='val_accuracy',mode='max')
ES = EarlyStopping(monitor='val_accuracy',min_delta=0,verbose=0,restore_best_weights = True,patience=10,mode='max')
RLP = ReduceLROnPlateau(monitor='val_loss',patience=10,factor=0.2,min_lr=0.0001)

In [ ]:
history = model.fit_generator(train_generator, 
                    epochs=200,  
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=[MCP,ES,RLP])
# history = model.fit_generator(train_generator,epochs=50,validation_data=validation_generator,verbose=1,callbacks=[MCP,ES,RLP])

In [ ]:
my_model = models.load_model('./Emogis_model.h5')

In [ ]:
print(train_generator.class_indices)

In [ ]:
output_map = {i:i+1 for i in range(7)}
print(output_map)